In [1]:
%reload_ext autoreload
%autoreload 2

In [5]:

import pandas as pd
from sampling import fetch_joined_data, WeightedSampler
from core import BaseCall, system_msg, user_msg
from pydantic import BaseModel, Field
from datetime import datetime
from typing import Optional, List

# Fetch and prepare data
joined_df = await fetch_joined_data()
joined_df['meeting_time'] = pd.to_datetime(joined_df['meeting_timestamp']).dt.strftime('%Y-%m-%d')

sampler = WeightedSampler(joined_df, date_column='meeting_time', decay_factor=0.2)
sampled_redults = sampler.sample(n_samples=100)  # mode defaults to 'recency'

def get_unique_value_counts(df, columns, format_output=False):
    """
    Returns unique values and their counts for specified columns.
    """
    result = {}
    
    for col in columns:
        try:
            value_counts = df[col].value_counts()
            value_dict_list = [{'value': value, 'count': count} 
                             for value, count in value_counts.items()]
            result[col] = value_dict_list
        except (TypeError, AttributeError):
            value_counts = df[col].astype(str).value_counts()
            value_dict_list = [{'value': value, 'count': count} 
                             for value, count in value_counts.items()]
            result[col] = value_dict_list
    
    if format_output:
        output_str = ""
        for col, value_list in result.items():
            output_str += f"\n{col}:\n"
            for item in value_list:
                output_str += f"  {item['value']}: {item['count']}\n"
        return output_str.strip()
    
    return result

# Get formatted string result for context
str_result = get_unique_value_counts(joined_df, ['topic_type', 'topic_name', 'speaker_name'], format_output=True)

class DateRange(BaseModel):
    start_date: Optional[datetime] = Field(None, description="Start date of the range")
    end_date: Optional[datetime] = Field(None, description="End date of the range")

class SearchFilter(BaseModel):
    topic_type: Optional[List[str]] = Field(None, description="List of topic types to filter by")
    topic_name: Optional[List[str]] = Field(None, description="List of topic names to filter by")
    speaker_name: Optional[List[str]] = Field(None, description="List of speaker names to filter by")
    date_range: Optional[DateRange] = Field(None, description="Date range to filter by")
    search_text: Optional[str] = Field(None, description="Text to search for in summary and details columns using similarity search")

def apply_search_filters(df: pd.DataFrame, filters: SearchFilter) -> pd.DataFrame:
    """
    Apply SearchFilter parameters to filter a DataFrame.
    """
    filtered_df = df.copy()
    
    if filters.topic_type:
        filtered_df = filtered_df[filtered_df['topic_type'].isin(filters.topic_type)]
    
    if filters.topic_name:
        filtered_df = filtered_df[filtered_df['topic_name'].isin(filters.topic_name)]
        
    if filters.speaker_name:
        filtered_df = filtered_df[filtered_df['speaker_name'].isin(filters.speaker_name)]
    
    if filters.date_range:
        if filters.date_range.start_date:
            filtered_df = filtered_df[
                pd.to_datetime(filtered_df['meeting_time']) >= filters.date_range.start_date
            ]
        if filters.date_range.end_date:
            filtered_df = filtered_df[
                pd.to_datetime(filtered_df['meeting_time']) <= filters.date_range.end_date
            ]
    
    if filters.search_text:
        text_mask = (
            filtered_df['summary'].str.contains(filters.search_text, case=False, na=False) |
            filtered_df['details'].str.contains(filters.search_text, case=False, na=False)
        )
        filtered_df = filtered_df[text_mask]
    
    return filtered_df

class SearchReflection(BaseModel):
    is_sufficient: bool = Field(..., description="Whether current results are sufficient to answer the user query")
    missing_aspects: List[str] = Field(..., description="List of aspects still missing from the results")
    suggested_filters: Optional[SearchFilter] = Field(None, description="Suggested additional filters if needed")
    reasoning: str = Field(..., description="Explanation of why results are/aren't sufficient")
    quality_score: float = Field(..., ge=0, le=1, description="Score indicating how well these results answer the query (0-1)")
    key_findings: List[str] = Field(..., description="List of key insights found in current results")

class SearchPlan(BaseModel):
    steps: List[str] = Field(..., description="List of search steps to execute")
    current_filters: SearchFilter = Field(..., description="Current step search parameters")
    is_final: bool = Field(..., description="Whether this is the final search step")
    feedback: str = Field(..., description="Analysis of current results and next steps needed")
    reflection: Optional[SearchReflection] = Field(None, description="Reflection on search results quality")

class SearchParams(BaseCall):
    plan: SearchPlan = Field(..., description="Search plan with current step and feedback")
    
    @classmethod
    async def extract(cls, user_query: str, table_context: str, sampled_df: pd.DataFrame, 
                     previous_results: Optional[pd.DataFrame] = None, 
                     current_results: Optional[pd.DataFrame] = None,
                     step_number: int = 1,
                     model: str = "gpt-4o-mini", 
                     use_cache: bool = False, 
                     force_store: bool = False):
        current_time = datetime.now()
        
        # Convert sampled results to a readable format
        sample_context = f"""Sample of recent records:
{sampled_df[['meeting_time', 'topic_type', 'topic_name', 'speaker_name']].head().to_markdown()}
"""
        
        # Add current results context if available
        results_context = ""
        if current_results is not None:
            results_context = f"""
Current search results:
{current_results[['meeting_time', 'topic_type', 'topic_name', 'speaker_name', 'summary']].head().to_markdown()}
Total results: {len(current_results)} entries
"""
        
        output = await cls.call([
            system_msg("""Plan and execute a multi-step search strategy to thoroughly answer the user's query.
                      Use the provided table context to identify and handle variations and possible misspellings.
                      
                      For each step:
                      1. Check the table context for all relevant variations of search terms
                      2. Include all valid variations in the search filters
                      3. Analyze current results if available
                      4. Reflect on whether results are sufficient to answer the query
                      5. Determine if additional search steps are needed
                      
                      When reflecting on results, consider:
                      - Do we have enough context about all mentioned entities?
                      - Are we capturing all relevant time periods?
                      - Have we found all important perspectives/opinions?
                      - Are the results specific enough to answer the query?
                      - Would additional filters help focus the results?
                      
                      Provide a quality score (0-1) based on:
                      - Relevance to the query
                      - Completeness of the answer
                      - Specificity of the results
                      - Coverage of different aspects
                      
                      Include key findings that summarize what we've learned from the results."""),
            user_msg(f"""Current datetime: {current_time}

Table context showing available values:
{table_context}

{sample_context}
{results_context}

User query: {user_query}
Current step: {step_number}

Plan the search strategy and reflect on current results.""")
        ], model=model, use_cache=use_cache, force_store=force_store)
        
        return output[0].plan

async def iterative_search(query: str, joined_df: pd.DataFrame, context: str, sampled_df: pd.DataFrame, verbose: bool = True):
    all_iterations = []  # Store all iteration results and their reflections
    results = None
    step = 1
    best_score = 0
    best_results = None
    
    while True:
        if verbose:
            print(f"\n=== Step {step} ===")
            
        # Get search plan for current step
        search_plan = await SearchParams.extract(
            user_query=query,
            table_context=context,
            sampled_df=sampled_df,
            previous_results=results,
            current_results=results,  # Pass current results for reflection
            step_number=step
        )
        
        if verbose:
            print("\nSearch Plan:")
            print(f"Steps: {search_plan.steps}")
            print(f"Current Filters: {search_plan.current_filters}")
            print(f"Feedback: {search_plan.feedback}")
            if search_plan.reflection:
                print("\nReflection:")
                print(f"Sufficient: {search_plan.reflection.is_sufficient}")
                print(f"Missing aspects: {search_plan.reflection.missing_aspects}")
                print(f"Quality Score: {search_plan.reflection.quality_score}")
                print(f"Key Findings: {search_plan.reflection.key_findings}")
                print(f"Reasoning: {search_plan.reflection.reasoning}")
        
        # Apply current filters
        current_results = apply_search_filters(joined_df, search_plan.current_filters)
        
        if verbose:
            print(f"\nFound {len(current_results)} results in this step")
        
        # Update results - handle list columns by converting to tuples
        if results is None:
            results = current_results
            if verbose:
                print("First step - using initial results")
        else:
            # Convert list columns to tuples for both DataFrames
            for df in [results, current_results]:
                for col in df.columns:
                    if df[col].apply(lambda x: isinstance(x, list)).any():
                        df[col] = df[col].apply(lambda x: tuple(x) if isinstance(x, list) else x)
            
            old_len = len(results)
            results = pd.concat([results, current_results]).drop_duplicates()
            new_len = len(results)
            
            if verbose:
                print(f"Added {new_len - old_len} new unique results")
        
        # Store current iteration results and reflection
        if search_plan.reflection:
            iteration_info = {
                'step': step,
                'results': results.copy(),
                'reflection': search_plan.reflection,
                'filters': search_plan.current_filters,
                'quality_score': search_plan.reflection.quality_score
            }
            all_iterations.append(iteration_info)
            
            # Update best results if current score is higher
            if search_plan.reflection.quality_score > best_score:
                best_score = search_plan.reflection.quality_score
                best_results = results.copy()
                
                if verbose:
                    print(f"\nNew best results found! Score: {best_score}")
        
        # Check if we're done based on reflection
        if search_plan.reflection and search_plan.reflection.is_sufficient:
            if verbose:
                print("\nSearch complete - results deemed sufficient")
            break
            
        step += 1
        if step > 5:  # Safety limit
            if verbose:
                print("\nSearch stopped - reached maximum steps (5)")
            break
    
    # Use best results found during iterations
    results = best_results if best_results is not None else results
    
    # Convert tuple columns back to lists in final results
    for col in results.columns:
        if results[col].apply(lambda x: isinstance(x, tuple)).any():
            results[col] = results[col].apply(lambda x: list(x) if isinstance(x, tuple) else x)
    
    # Calculate relevance scores and sort results
    search_terms = set([term.lower() for term in query.split()])
    
    def calculate_relevance(row):
        text = f"{row['summary']} {row['details']}".lower()
        # Count occurrences of search terms
        term_matches = sum(text.count(term) for term in search_terms)
        # Boost score for more recent dates
        recency_boost = pd.to_datetime(row['meeting_time']).timestamp() / 1e9
        return term_matches + (recency_boost / 1e11)  # Normalize recency boost
    
    results['relevance_score'] = results.apply(calculate_relevance, axis=1)
    results = results.sort_values('relevance_score', ascending=False).drop(columns=['relevance_score'])
    
    if verbose:
        print(f"\nFinal Results: {len(results)} total unique entries")
        print(f"Best Quality Score: {best_score}")
        print("\nKey Findings Across Iterations:")
        for iteration in all_iterations:
            print(f"\nStep {iteration['step']} (Score: {iteration['quality_score']}):")
            for finding in iteration['reflection'].key_findings:
                print(f"- {finding}")
    
    return results, all_iterations

# Example usage
results, search_history = await iterative_search(
    query="what users say about vexa, not covorkers",
    joined_df=joined_df,
    context=str_result,
    sampled_df=sampled_redults,
    verbose=True
)


=== Step 1 ===

Search Plan:
Steps: ["Identify relevant variations of 'Vexa' and 'user feedback'", 'Search for user feedback on Vexa', 'Analyze the results for insights on user opinions about Vexa']
Current Filters: topic_type=None topic_name=['Vexa', 'VEXA', 'VEX.AI'] speaker_name=None date_range=None search_text='user feedback'
Feedback: The initial search will focus on gathering user feedback specifically about Vexa, excluding coworker opinions. This will help in understanding the general sentiment and experiences of users with the product.

Reflection:
Sufficient: False
Missing aspects: ['Specific user testimonials', 'Comparative analysis with competitors', 'User demographics']
Quality Score: 0.4
Key Findings: ['Initial search focused on user feedback for Vexa but needs refinement to exclude coworker insights.']
Reasoning: The current results may not provide a comprehensive view of user feedback as they might include coworker opinions or lack specific user testimonials.

Found 1 r

/tmp/ipykernel_395407/1211616517.py:223: FutureWarning: 'any' with datetime64 dtypes is deprecated and will raise in a future version. Use (obj != pd.Timestamp(0)).any() instead.
  if df[col].apply(lambda x: isinstance(x, list)).any():



Search Plan:
Steps: ["Identify variations of 'Vexa' and 'user feedback'", 'Search for user feedback on Vexa', 'Analyze results for relevance and completeness']
Current Filters: topic_type=['product', 'feedback', 'discussion'] topic_name=['Vexa'] speaker_name=None date_range=None search_text='user feedback'
Feedback: Searching for user feedback specifically about Vexa, excluding coworker opinions. The current results are limited, with only one entry related to Vexa. We need to broaden the search to capture more user perspectives.

Found 1 results in this step
Added 0 new unique results

=== Step 4 ===

Search Plan:
Steps: ["Identify variations of 'Vexa' and 'user feedback'", 'Search for user feedback on Vexa', 'Analyze results for relevance and completeness', 'Determine if further searches are needed']
Current Filters: topic_type=None topic_name=['Vexa'] speaker_name=None date_range=None search_text='user feedback'
Feedback: Current results are limited to a single entry that discusses 

In [6]:
results

,summary_index,summary,details,referenced_text,topic_name,topic_type,meeting_id,meeting_timestamp,speaker_name,other_speakers,meeting_time
676,6,"Vexa is a product discussed in the meeting, fo...",Vexa is currently in the testing phase with a ...,"Dmitry Grankin: Yeah, we went on marketing, b...",Vexa,product,70cd7290-801a-4caf-9786-359cc6e16c60,2024-09-30 10:01:22.780,Dmitry Grankin,[Umar Lateef],2024-09-30


In [12]:
for step in search_history:
    print(step['filters'])
    print(step['reflection'])

topic_type=None topic_name=['Vexa', 'VEXA', 'VEX.AI'] speaker_name=None date_range=None search_text='user feedback'
is_sufficient=False missing_aspects=['Specific user testimonials', 'Comparative analysis with competitors', 'User demographics'] suggested_filters=SearchFilter(topic_type=['feedback', 'discussion', 'concern'], topic_name=None, speaker_name=None, date_range=None, search_text=None) reasoning='The current results may not provide a comprehensive view of user feedback as they might include coworker opinions or lack specific user testimonials.' quality_score=0.4 key_findings=['Initial search focused on user feedback for Vexa but needs refinement to exclude coworker insights.']
topic_type=['feedback', 'discussion', 'concern'] topic_name=['Vexa', 'VEXA', 'VEX.AI'] speaker_name=None date_range=None search_text='user feedback'
is_sufficient=False missing_aspects=['User feedback from external sources', "Diverse opinions on Vexa's features", 'Comparative feedback with similar product

In [14]:
step['results']

,summary_index,summary,details,referenced_text,topic_name,topic_type,meeting_id,meeting_timestamp,speaker_name,other_speakers,meeting_time
676,6,"Vexa is a product discussed in the meeting, fo...",Vexa is currently in the testing phase with a ...,"Dmitry Grankin: Yeah, we went on marketing, b...",Vexa,product,70cd7290-801a-4caf-9786-359cc6e16c60,2024-09-30 10:01:22.780,Dmitry Grankin,"(Umar Lateef,)",2024-09-30


In [16]:
step['reflection']

SearchReflection(is_sufficient=False, missing_aspects=['User feedback from external sources', "Diverse opinions on Vexa's features", 'Comparative feedback with similar products'], suggested_filters=SearchFilter(topic_type=['feedback', 'discussion', 'concern'], topic_name=['Vexa', 'VEXA', 'VEX.AI'], speaker_name=None, date_range=None, search_text=None), reasoning='The current results do not provide sufficient user feedback on Vexa. We need to expand the search to include more user-generated content and feedback from various sources.', quality_score=0.2, key_findings=['Current results focus on Vexa as a product but lack user feedback.', 'Need to capture external user opinions and experiences with Vexa.'])

In [19]:
pd.concat([step['results'] for step in search_history])



,summary_index,summary,details,referenced_text,topic_name,topic_type,meeting_id,meeting_timestamp,speaker_name,other_speakers,meeting_time
676,6,"Vexa is a product discussed in the meeting, fo...",Vexa is currently in the testing phase with a ...,"Dmitry Grankin: Yeah, we went on marketing, b...",Vexa,product,70cd7290-801a-4caf-9786-359cc6e16c60,2024-09-30 10:01:22.780,Dmitry Grankin,[Umar Lateef],2024-09-30
676,6,"Vexa is a product discussed in the meeting, fo...",Vexa is currently in the testing phase with a ...,"Dmitry Grankin: Yeah, we went on marketing, b...",Vexa,product,70cd7290-801a-4caf-9786-359cc6e16c60,2024-09-30 10:01:22.780,Dmitry Grankin,"(Umar Lateef,)",2024-09-30
